# 310 - Experiment #1 - Hierarchical NER with reference dataset

Requirements : 
* Create datasets in `m2_joint-labelling_for_ner` : `M2_200-prepare_datasets_joint_labelling`

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

1


In [2]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [3]:
#!pip install --upgrade transformers datasets spacy transformers[sentencepiece] seqeval

## Initialisation
Set the BASE path.
If run on Google Colab, will also mout Google Drive to the moutpoint given below

In [4]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  DATA_BASE = Path('/work/stual/res_ICDAR/method_2')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_3')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m3_hierarchical_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m3_hierarchical_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_3


## Constants

In [5]:
RUN_CAMEMBERT_IO = True
RUN_CAMEMBERT_IOB2 = False
#Can't run together because of convert_tokenizer_
RUN_PTRN_CAMEMBERT_IO = False
RUN_PTRN_CAMEMBERT_IOB2 = False

# Number of times a model will be trained & evaluated on each a dataset
N_RUNS = 5

## CamemBERT - Config

In [6]:
# COMMON CONSTANTS

TRAINING_CONFIG = {
    "evaluation_strategy": "steps",
    "eval_steps": 100,
    "max_steps": 5000,
    "learning_rate": 1e-4,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "weight_decay": 1e-5,
    "load_best_model_at_end": True,
    "greater_is_better":True,
    "metric_for_best_model": "f1",
    "save_strategy": "steps",
    "save_steps": 100,
    "save_total_limit": 1
}

In [7]:
import pandas as pd
from datasets import load_from_disk

#Print examples from datasets
def loadExample(INPUT_DIR,set_length:int,i:int,subset:str):
    set_ = load_from_disk(INPUT_DIR / f"huggingface_{set_length}")
    data = {"tokens": set_[subset][i]["tokens"],
            "labels": set_[subset][i]["ner_tags"]}
    df = pd.DataFrame.from_dict(data)
    print(df)

## Hierarchical NER : Trees

### Tree with IO Labels

In [8]:
from hierarchicalNER.trees import Ltree, Wtree
Ltree.pretty_print(unicodelines=True, nodedist=2)
print(Wtree)

                                                             EN                                                                                        
 ┌──────────────┬───────────────────────┬────────────────────┼────────────────────────────────────────┬───────────────────────────────────────────┐      
 O            I-PER                   I-ACT                I-DESC                                   I-SPAT                                     I-TITRE 
 │      ┌───────┴──────────┐            │        ┌───────────┼───────────────┐            ┌───────────┼───────────────┬───────────────┐           │      
O+O  I-PER+O         I-PER+i_TITREH  I-ACT+O  I-DESC+O  I-DESC+i_ACT  I-DESC+i_TITREP  I-SPAT+O  I-SPAT+i_LOC  I-SPAT+i_CARDINA  I-SPAT+i_FT  I-TITRE+O
                                                                                                                      L                                

(-100.0
  (1.0 1.0)
  (1.0 1.0 1.0)
  (1.0 1.0)
  (1.0 1.0 1.0 1.0)
  (1.0 1.0 1.0 

### Tree with IOB2 Labels

In [9]:
from hierarchicalNER.trees import Ltree_iob2, Wtree_iob2
Ltree_iob2.pretty_print(unicodelines=True, nodedist=2)
print(Wtree_iob2)

                                                                                                                                                                                                                              EN                                                                                                                                                                                                                                               
 ┌───────────────────────────────────────┬───────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────┐                  
 O                                      PER                           

## 311 - Train & eval : IO Ref dataset with CamemBERT model

In [10]:
MODEL_NAME = "camembert_ner"
MODEL = "Jean-Baptiste/camembert-ner"
LABEL = "io"
FOLDER = "311-camembert-ner-hierarchical-loss-io"

### 311.1 Load IO dataset

In [11]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_1_prepared_dataset_ref_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-310-experiment_1_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_1_prepared_dataset_ref_io_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-310-experiment_1_metrics'))

In [12]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens             labels
0  Duffau-Pauillac            I-PER+O
1                (            I-PER+O
2              Chs            I-PER+O
3                )            I-PER+O
4                ,                O+O
5          Enghien       I-SPAT+i_LOC
6                ,           I-SPAT+O
7               16  I-SPAT+i_CARDINAL
8                .                O+O
9                *                O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 311.2 Fine-tuning with IO labels - train & eval

In [13]:
from config import logger
from datasets import load_from_disk
import json
from hierarchicalNER.util_IO import init_model, train_eval_loop

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")
            
            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

2023-02-06 15:53:17.082002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 15:53:18.991198: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-06 15:53:18.991311: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-06 15:53:18.991322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Tree positions of all the leaves{'O+O': (0, 0), 'I-PER+O': (1, 0), 'I-PER+i_TITREH': (1, 1), 'I-ACT+O': (2, 0), 'I-DESC+O': (3, 0), 'I-DESC+i_ACT': (3, 1), 'I-DESC+i_TITREP': (3, 2), 'I-SPAT+O': (4, 0), 'I-SPAT+i_LOC': (4, 1), 'I-SPAT+i_CARDINAL': (4, 2), 'I-SPAT+i_FT': (4, 3), 'I-TITRE+O': (5, 0)} #1
Num of classes : 12


[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
import time
import datetime
from hierarchicalNER.util_IO import _convert_tokenizer

if RUN_CAMEMBERT_IO:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at Jean-Baptiste/camembert-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([12, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([12]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set don't have a corresponding argument in `

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Precision-l1,Recall-l1,F1-l1,Accuracy-l1,Precision-l2,Recall-l2,F1-l2,Accuracy-l2,Precision-all,Recall-all,F1-all,Accuracy-all,Precision-das,Recall-das,F1-das,Accuracy-das,Per,Act,Act L1,Act L2,Desc,Titreh,Titrep,Spat,Loc,Cardinal,Ft,Titre
100,No log,0.341509,0.938622,0.938894,0.938758,0.940045,0.905488,0.935433,0.920217,0.945010,0.964997,0.937333,0.950964,0.972195,0.930803,0.936270,0.933529,0.958602,0.930018,0.940760,0.935358,0.953327,"{'precision': 0.9219653179190751, 'recall': 0.9437869822485208, 'f1': 0.932748538011696, 'number': 676}","{'precision': 0.8462897526501767, 'recall': 0.9229287090558767, 'f1': 0.8829493087557604, 'number': 519}","{'precision': 0.8462897526501767, 'recall': 0.952286282306163, 'f1': 0.8961646398503273, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}","{'precision': 0.9366197183098591, 'recall': 0.9582132564841499, 'f1': 0.9472934472934472, 'number': 694}","{'precision': 0.9445876288659794, 'recall': 0.9632063074901446, 'f1': 0.9538061158100194, 'number': 761}","{'precision': 0.9882525697503671, 'recall': 0.9926253687315634, 'f1': 0.9904341427520236, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
200,No log,0.219527,0.946870,0.954822,0.950829,0.949479,0.934805,0.955906,0.945237,0.954444,0.956405,0.950667,0.953527,0.974553,0.944170,0.953598,0.948860,0.964499,0.945321,0.959762,0.952487,0.962761,"{'precision': 0.9604105571847508, 'recall': 0.9689349112426036, 'f1': 0.9646539027982327, 'number': 676}","{'precision': 0.8987341772151899, 'recall': 0.9576107899807321, 'f1': 0.9272388059701492, 'number': 519}","{'precision': 0.8987341772151899, 'recall': 0.9880715705765407, 'f1': 0.9412878787878788, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}","{'precision': 0.7916666666666666, 'recall': 0.5757575757575758, 'f1': 0.6666666666666667, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}","{'precision': 0.9449152542372882, 'recall': 0.9639769452449568, 'f1': 0.9543509272467903, 'number': 694}","{'precision': 0.9325699745547074, 'recall': 0.9632063074901446, 'f1': 0.9476405946994182, 'number': 761}","{'precision': 0.9897209985315712, 'recall': 0.9941002949852508, 'f1': 0.9919058130978661, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
300,No log,0.176005,0.958995,0.961772,0.960382,0.960899,0.937500,0.952756,0.945066,0.964871,0.965703,0.957333,0.961500,0.976663,0.949752,0.954772,0.952255,0.970767,0.962880,0.966468,0.964671,0.973312,"{'precision': 0.9693430656934306, 'recall': 0.9822485207100592, 'f1': 0.9757531227038942, 'number': 676}","{'precision': 0.9288461538461539, 'recall': 0.930635838150289, 'f1': 0.9297401347449471, 'number': 519}","{'precision': 0.9288461538461539, 'recall': 0.9602385685884692, 'f1': 0.9442815249266862, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.21875, 'recall': 0.23333333333333334, 'f1': 0.22580645161290322, 'number': 30}","{'precision': 0.8648648648648649, 'recall': 0.9696969696969697, 'f1': 0.9142857142857143, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}","{'precision': 0.9456366237482118, 'recall': 0.952449567723343, 'f1': 0.9490308686288585, 'number': 694}","{'precision': 0.9504563233376793, 'recall': 0.9579500657030223, 'f1': 0.9541884816753926, 'number': 761}","{'precision': 0.9882869692532943, 'recall': 0.995575221238938, 'f1': 0.9919177075679647, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 

The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 676
  Batch size = 16
/lrde/home2/stual/stage_DAS/m3_hierarchical_ner/hierarchicalNER/util_IO.py:175: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision 

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.9604105571847508, 'recall': 0.9689349112426036, 'f1': 0.9646539027982327, 'number': 676}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8987341772151899, 'recall': 0.9576107899807321, 'f1': 0.9272388059701492, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8987341772151899, 'recall'

Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}" of type <class 'dict'> for key "eval/TITREP" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9456366237482118, 'recall': 0.952449567723343, 'f1': 0.9490308686288585, 'number': 694}" of type <class 'dict'> for key "eval/SPAT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9504563233376793, 'recall': 0.9579500657030223, 'f1': 0.9541884816753926, 'number': 761}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9882869692532943, 'recall': 0.995575221238938, 'f1': 0.9919177075679647, 'number': 678}" of t

...

Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}" of type <class 'dict'> for key "eval/TITRE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /work/stual/res_ICDAR/method_3/tmp/311-camembert-ner-hierarchical-loss-io/checkpoint-1500
Configuration saved in /work/stual/res_ICDAR/method_3/tmp/311-camembert-ner-hierarchical-loss-io/checkpoint-1500/config.json
Model weights saved in /work/stual/res_ICDAR/method_3/tmp/311-camembert-ner-hierarchical-loss-io/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in /work/stual/res_ICDAR/method_3/tmp/311-camembert-ner-hierarchical-loss-io/checkpoint-1500/tokenizer_config.json
Special tokens file saved in /work/stual/res_ICDAR/method_3/tmp/311-camembert-ner-hierarchical-loss-io/checkpoint-1500/special_tokens_map.json
Deleting older checkpoint [/work/stual/res_ICDAR/method_3/tmp/311-camembert-ner-hierarch

Trainer is attempting to log a value of "{'precision': 0.9765739385065886, 'recall': 0.9866863905325444, 'f1': 0.9816041206769685, 'number': 676}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9230769230769231, 'recall': 0.9479768786127167, 'f1': 0.935361216730038, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9528487229862476, 'recall': 0.9642147117296223, 'f1': 0.958498023715415, 'number': 503}" of type <class 'dict'> for key "eval/ACT_L1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.2916666666666667, 'recall': 0.4375, 'f1': 0.3500

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Trainer is attempting to log a value of "{'precision': 0.9786729857819905, 'recall': 0.9804154302670623, 'f1': 0.9795434331455677, 'number': 1685}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8995555555555556, 'recall': 0.9250457038391224, 'f1': 0.9121225777377197, 'number': 1094}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9691848906560636, 'recall': 0.9456838021338506, 'f1': 0.9572901325478645, 'number': 1031}" of type <class 'dict'> for key "eval/ACT_L1" as

Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}" of type <class 'dict'> for key "eval/TITRE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Run-time is equal to 0:13:27.984950


## 312 - Train & eval : IOB2 Ref dataset with CamemBERT model

In [15]:
MODEL_NAME = "camembert_ner"
MODEL = "Jean-Baptiste/camembert-ner"
LABEL = "iob2"
FOLDER = "312-camembert-ner-hierarchical-loss-iob2"

### 312.1 Load IOB2 dataset

In [16]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_1_prepared_dataset_ref_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-310-experiment_1_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_1_prepared_dataset_ref_iob2_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-310-experiment_1_metrics'))

In [17]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens               labels
0  Duffau-Pauillac            I-b_PER+O
1                (            I-i_PER+O
2              Chs            I-i_PER+O
3                )            I-i_PER+O
4                ,                  O+O
5          Enghien       I-b_SPAT+b_LOC
6                ,           I-i_SPAT+O
7               16  I-i_SPAT+b_CARDINAL
8                .                  O+O
9                *                  O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 312.2 Fine-tuning with IOB2 labels - train & eval

In [18]:
from config import logger
from datasets import load_from_disk
from hierarchicalNER.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer
import json

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")
            
            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

Tree positions of all the leaves{'O+O': (0, 0, 0), 'I-b_PER+O': (1, 0, 0), 'I-i_PER+O': (1, 0, 1), 'I-b_PER+b_TITREH': (1, 1, 0), 'I-i_PER+b_TITREH': (1, 1, 1), 'I-i_PER+i_TITREH': (1, 1, 2), 'I-b_ACT+O': (2, 0, 0), 'I-i_ACT+O': (2, 0, 1), 'I-b_DESC+O': (3, 0, 0), 'I-i_DESC+O': (3, 0, 1), 'I-b_DESC+b_ACT': (3, 1, 0), 'I-i_DESC+b_ACT': (3, 1, 1), 'I-i_DESC+i_ACT': (3, 1, 2), 'I-b_DESC+b_TITREP': (3, 2, 0), 'I-i_DESC+b_TITREP': (3, 2, 1), 'I-i_DESC+i_TITREP': (3, 2, 2), 'I-b_SPAT+O': (4, 0, 0), 'I-i_SPAT+O': (4, 0, 1), 'I-b_SPAT+b_LOC': (4, 1, 0), 'I-i_SPAT+b_LOC': (4, 1, 1), 'I-i_SPAT+i_LOC': (4, 1, 2), 'I-b_SPAT+b_CARDINAL': (4, 2, 0), 'I-i_SPAT+b_CARDINAL': (4, 2, 1), 'I-i_SPAT+i_CARDINAL': (4, 2, 2), 'I-b_SPAT+b_FT': (4, 3, 0), 'I-i_SPAT+b_FT': (4, 3, 1), 'I-i_SPAT+i_FT': (4, 3, 2), 'I-b_TITRE+O': (5, 0, 0), 'I-i_TITRE+O': (5, 0, 1)} #1
Num of classes : 29


loading file sentencepiece.bpe.model from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/sentencepiece.bpe.model
loading file tokenizer.json from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/special_tokens_map.json
loading file tokenizer_config.json from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/tokenizer_config.json
[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Packag

In [19]:
import time
import datetime

if RUN_CAMEMBERT_IOB2:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IOB2 labels")

Skipped finetuning model for IOB2 labels


## 313 - Train & eval : IO Ref dataset with Pretrained CamemBERT model

In [20]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "io"
FOLDER = "313-pretrained-camembert-ner-hierarchical-loss-io"

### 313.1 Load IO dataset

In [21]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_1_prepared_dataset_ref_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-310-experiment_1_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_1_prepared_dataset_ref_io_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-310-experiment_1_metrics'))

In [22]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens             labels
0  Duffau-Pauillac            I-PER+O
1                (            I-PER+O
2              Chs            I-PER+O
3                )            I-PER+O
4                ,                O+O
5          Enghien       I-SPAT+i_LOC
6                ,           I-SPAT+O
7               16  I-SPAT+i_CARDINAL
8                .                O+O
9                *                O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 313.2 Fine-tuning with IO labels - train & eval

In [23]:
from config import logger
from datasets import load_from_disk
from hierarchicalNER.util_IO import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [24]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IO:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
        
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning pretrained model for IO labels")

Skipped finetuning pretrained model for IO labels


## 314 - Train & eval : IOB2 Ref dataset with Pretrained CamemBERT model

In [25]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "iob2"
FOLDER = "314-pretrained-camembert-ner-hierarchical-loss-iob2"

### 314.1 Load IOB2 dataset

In [26]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_1_prepared_dataset_ref_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-310-experiment_1_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_1_prepared_dataset_ref_iob2_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-310-experiment_1_metrics'))

In [27]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


            tokens               labels
0  Duffau-Pauillac            I-b_PER+O
1                (            I-i_PER+O
2              Chs            I-i_PER+O
3                )            I-i_PER+O
4                ,                  O+O
5          Enghien       I-b_SPAT+b_LOC
6                ,           I-i_SPAT+O
7               16  I-i_SPAT+b_CARDINAL
8                .                  O+O
9                *                  O+O


In [28]:
from config import logger
from datasets import load_from_disk
from hierarchicalNER.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer
import json

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
            
            torch.cuda.empty_cache()

In [29]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IOB2:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning pretrained model for IOB2 labels")

Skipped finetuning pretrained model for IOB2 labels
